In [103]:
# importando pacotes
from ucimlrepo import fetch_ucirepo 
import pandas
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy

In [104]:
# CARREGANDO DADOS

# importando dataset
dataset = fetch_ucirepo(id=53) # 150 registros e 4 atributos
  
# coletando as informações
data_frame = dataset.data.original

In [105]:
# TRATANDO DADOS

print("PRÉ TRATAMENTO: ", len(data_frame))

# removendo colunas com muitos nulos
tolerancia = len(data_frame) * 0.7
data_frame = data_frame.dropna(axis=1, thresh=tolerancia)

# removendo duplicados
print("VALORES DUPLICADOS: ", data_frame.duplicated().sum())
data_frame = data_frame.drop_duplicates()

# convertendo colunas categóricas em valores inteiros
conversores = {}
for coluna in data_frame.columns:
    if (data_frame[coluna].dtype == type(object)):
        conversor = LabelEncoder()
        data_frame[coluna] = conversor.fit_transform(data_frame[coluna])
        conversores[coluna] = conversor

print("PÓS TRATAMENTO: ", len(data_frame))

PRÉ TRATAMENTO:  150
VALORES DUPLICADOS:  3
PÓS TRATAMENTO:  147


In [106]:
print(data_frame.columns)

Index(['sepal length', 'sepal width', 'petal length', 'petal width', 'class'], dtype='object')


In [107]:
# DIVIDINDO O DATASET TRATADO

atributos = data_frame.drop(["class"], axis=1)
respostas = data_frame[["class"]]

a_treino, a_teste, r_treino, r_teste = train_test_split(atributos, respostas, test_size=0.3, random_state=42)

# convertendo de (N, 1) para (N,)
r_treino = numpy.ravel(r_treino)
r_teste = numpy.ravel(r_teste)


In [108]:
print("TAMANHO DO DATASET TRATADO: ", len(data_frame))
print("CONJUNTO DE TREINO: ", a_treino.shape, " e ", r_treino.shape)
print("CONJUNTO DE TESTE: ", a_teste.shape, " e ", r_teste.shape)

TAMANHO DO DATASET TRATADO:  147
CONJUNTO DE TREINO:  (102, 4)  e  (102,)
CONJUNTO DE TESTE:  (45, 4)  e  (45,)


In [109]:
# CRIANDO PARÂMETROS PARA OS MODELOS

lista_parametros = [
    {"id": "1", "n_estimators": 10, "criterion": "gini", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "2", "n_estimators": 100, "criterion": "gini", "max_depth": 10, "min_samples_split": 3, "min_samples_leaf": 10},
    {"id": "3", "n_estimators": 10, "criterion": "entropy", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "4", "n_estimators": 100, "criterion": "entropy", "max_depth": 10, "min_samples_split": 3, "min_samples_leaf": 10},
]

In [110]:
# APLICANDO MODELO

# criando a lista de resultados finais
lista_resultados = []

for parametros in lista_parametros:
    # criando classificador
    classificador = RandomForestClassifier(
        n_estimators=parametros["n_estimators"], 
        criterion=parametros["criterion"], 
        max_depth=parametros["max_depth"], 
        min_samples_split=parametros["min_samples_split"], 
        min_samples_leaf=parametros["min_samples_leaf"], 
        random_state=42
    )

    # treinando o modelo
    classificador.fit(a_treino, r_treino)

    # prevendo respostas
    r_previsao = classificador.predict(a_teste)

    # calculando métricas
    acuracia = accuracy_score(r_teste, r_previsao)
    f1 = f1_score(r_teste, r_previsao, average="weighted")

    # salvando resultados
    lista_resultados.append([parametros["id"], acuracia, f1])

In [111]:
# mostrando resultados
tabela_resultados = pandas.DataFrame(lista_resultados, columns=["Identificador", "Acurácia", "F1-score"])
print(tabela_resultados)

  Identificador  Acurácia  F1-score
0             1  0.955556  0.955556
1             2  0.955556  0.955556
2             3  0.955556  0.955556
3             4  0.955556  0.955556
